# Protein Folding using Variational Quantum Eigensolver (VQE)

Protein folding has been one of the most interesting areas of study since 1960 and is one of the hardest problem to solve in biochemistry. Proteins are made of long chains of amino acids that are present in all living organism. They have varying range of functions from providing physical structures to allow muscles to grow and bend. Protein folds distinctively, and with varying chains of amino acids it becomes more difficult and complex to find protein's stable structure. 

Researchers at IBM have developed a quantum algorithm to demonstrate how quantum computing can help improve these computations to find protein structures more efficiently. In this notebook, we will demonstrate how to use Variational Quantum Eigensolver (VQE) algorithm using Qiskit Nature to predict protein structures. We will demonstrate how we can run this algorithm on Amazon Braket using Amazon Braket's hybrid jobs. 

We will begin by importing Amazon Braket SDK to run jobs locally or as a hybrid job on Amazon Braket. 

In [ ]:
## NOTE: Optional
# !pip install -r requirements.txt -q 

In [ ]:
from braket.jobs.local.local_job import LocalQuantumJob
from braket.aws import AwsQuantumJob, AwsSession
from braket.jobs.config import InstanceConfig
import time
import boto3

Amazon Braket allows you to bring any third party libraries by allowing you to build your own docker image. We will build a Docker image by extending base Amazon Braket image and installing Qiskit Nature and Qiskit Braket provider plugin.

In [ ]:
%%writefile Dockerfile
FROM 292282985366.dkr.ecr.us-west-2.amazonaws.com/amazon-braket-base-jobs:1.0-cpu-py37-ubuntu18.04
COPY requirements.txt ./
RUN python3 -m pip install -r ./requirements.txt

Docker login to Amazon Braket's ECR repository to pull base Docker image. Do not change region or account id below.

In [ ]:
%%sh
#!/bin/bash
region="us-west-2"
account="292282985366"
aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $account.dkr.ecr.$region.amazonaws.com

Next, we will allow Docker login to your AWS Account. Here, update your region and account id where you would like to run Amazon Braket job.

In [ ]:
%%sh
#!/bin/bash
region="us-west-2"
account="123456789012"
aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $account.dkr.ecr.$region.amazonaws.com

Next, we will create AWS ECR repository where we can push our Docker image. 

In [ ]:
%%sh
#!/bin/bash
region="us-west-2"
aws ecr create-repository --repository-name amazon-braket-protein-folding --region $region

After successfully creating AWS ECR repository to store docker image, we will go ahead and build, tag and push docker image to AWS ECR repository.

In [ ]:
! docker build -t amazon-braket-protein-folding .

In [ ]:
%%sh
#!/bin/bash
region="us-west-2"
account="123456789012"
docker tag amazon-braket-protein-folding:latest $account.dkr.ecr.$region.amazonaws.com/amazon-braket-protein-folding:latest

In [ ]:
%%sh
#!/bin/bash
region="us-west-2"
account="123456789012"
docker push $account.dkr.ecr.$region.amazonaws.com/amazon-braket-protein-folding:latest

In [ ]:
penalty_params = {
    "penalty_back": 10,
    "penalty_chiral": 10,
    "penalty_1": 10
}

optimizer_params = {
    "maxiter": 500
}

hyperparameters_quantum_job = {
    "main_chain": "APRLRFY", 
    "side_chains": None,               
    "penalty_params": penalty_params,
    "optimizer": "COBYLA",
    "optimizer_params": optimizer_params,
    "interaction" : "MiyazawaJerniganInteraction",
    "shots": 10
}

### Run it locally

In [ ]:
from braket.jobs.local import LocalQuantumJob

account = "123456789012"
region = "us-west-2"

job = LocalQuantumJob.create(
    device="local",
    source_module="src/vqe.py",
    hyperparameters=hyperparameters_quantum_job,
    image_uri=f"{account}.dkr.ecr.{region}.amazonaws.com/amazon-braket-protein-folding:latest"
)

### Submit Amazon Braket Hybrid job

In [ ]:
q_device = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
job_instance_type="ml.m5.4xlarge"

account = "123456789012"
region = "us-west-2"

## Run it as a job on Braket. Provide quantum hardware ARN to run on a quantum device
job = AwsQuantumJob.create(
    device=q_device,
    source_module="src/vqe.py",
    job_name=f"protein-folding-job-{int(time.time())}",
    instance_config=InstanceConfig(instanceType=job_instance_type),
    image_uri=f"{account}.dkr.ecr.{region}.amazonaws.com/amazon-braket-protein-folding:latest",
    copy_checkpoints_from_job=None,
    hyperparameters=hyperparameters_quantum_job,
    wait_until_complete=False
)

After successfully submit, you can monitor progress of the job and track VQE algorithm finding minimum energy of the protein. Energy graph is available under "Monitor" tab in Amazon Braket job. 

![Energy Graph](energy_graph.png)

 <center>Energy Graph</center>

### Analyzing result

In [ ]:
# We will download our job result
job.download_result()

In [ ]:
# Extract files
!tar -xf model.tar.gz

At this point, you should see 3 files extracted from `model.tar.gz`. 
1. Your provided chain name XYZ coordinates file
2. PDB file
3. optimal_point.npy file containing optimal point for minimum energy. Next time you can provide this optimal point to begin optimizing from these points. 

Navigate to `https://www.rcsb.org/3d-view` and click on `Open Files`. Select either PDB or xyz file to visualize the structure. 

## Citations

Qiskit: An Open-Source Framework for Quantum Computing. 2021, https://doi.org10.5281/zenodo.2573505.